# Eksploracyjna Analiza Danych (EDA) - Klasyfikacja Leków

Ten notebook zawiera wstępną analizę danych dotyczących leków i pacjentów.

In [ ]:
# Import bibliotek
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Konfiguracja wizualizacji
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Biblioteki załadowane pomyślnie!")

## 1. Wczytanie danych

In [ ]:
# Wczytanie danych
data = pd.read_csv('../data/real_drug_dataset.csv', encoding='utf-8-sig')

# Usunięcie BOM z nazw kolumn jeśli występuje
data.columns = [col.replace('\ufeff', '') for col in data.columns]

print(f"Wymiary danych: {data.shape}")
print(f"\nKolumny: {list(data.columns)}")
data.head()

## 2. Podstawowe informacje o danych

In [ ]:
# Informacje o typach danych
print("Informacje o danych:")
print(data.info())
print("\n" + "="*60)
print("\nStatystyki opisowe dla zmiennych numerycznych:")
print(data.describe())

In [ ]:
# Sprawdzenie brakujących wartości
print("Brakujące wartości:")
missing = data.isnull().sum()
print(missing[missing > 0])
if missing.sum() == 0:
    print("Brak brakujących wartości w zbiorze danych!")

## 3. Analiza zmiennych kategorycznych

In [ ]:
# Analiza zmiennych kategorycznych
categorical_cols = ['Gender', 'Condition', 'Drug_Name', 'Side_Effects']

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, col in enumerate(categorical_cols):
    value_counts = data[col].value_counts()
    axes[idx].barh(range(len(value_counts)), value_counts.values)
    axes[idx].set_yticks(range(len(value_counts)))
    axes[idx].set_yticklabels(value_counts.index)
    axes[idx].set_xlabel('Liczba wystąpień')
    axes[idx].set_title(f'Rozkład {col}')
    axes[idx].invert_yaxis()

plt.tight_layout()
plt.savefig('../results/categorical_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

# Wyświetlenie liczebności
for col in categorical_cols:
    print(f"\n{col}:")
    print(data[col].value_counts())

## 4. Analiza zmiennych numerycznych

In [ ]:
# Rozkłady zmiennych numerycznych
numerical_cols = ['Age', 'Dosage_mg', 'Treatment_Duration_days', 'Improvement_Score']

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, col in enumerate(numerical_cols):
    axes[idx].hist(data[col], bins=30, edgecolor='black', alpha=0.7)
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Częstość')
    axes[idx].set_title(f'Rozkład {col}')
    axes[idx].axvline(data[col].mean(), color='red', linestyle='--', label=f'Średnia: {data[col].mean():.2f}')
    axes[idx].axvline(data[col].median(), color='green', linestyle='--', label=f'Mediana: {data[col].median():.2f}')
    axes[idx].legend()

plt.tight_layout()
plt.savefig('../results/numerical_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Box plots dla zmiennych numerycznych
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, col in enumerate(numerical_cols):
    axes[idx].boxplot(data[col], vert=True)
    axes[idx].set_ylabel(col)
    axes[idx].set_title(f'Box Plot - {col}')

plt.tight_layout()
plt.savefig('../results/boxplots.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Analiza korelacji

In [ ]:
# Macierz korelacji dla zmiennych numerycznych
correlation_matrix = data[numerical_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Macierz korelacji zmiennych numerycznych')
plt.tight_layout()
plt.savefig('../results/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("Macierz korelacji:")
print(correlation_matrix)

## 6. Analiza zmiennej docelowej (Drug_Name)

In [ ]:
# Rozkład leków (zmienna docelowa)
drug_counts = data['Drug_Name'].value_counts()

plt.figure(figsize=(14, 8))
drug_counts.plot(kind='barh')
plt.xlabel('Liczba wystąpień')
plt.ylabel('Nazwa leku')
plt.title('Rozkład leków w zbiorze danych')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('../results/drug_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Liczba unikalnych leków: {data['Drug_Name'].nunique()}")
print(f"\nRozkład leków:")
print(drug_counts)

## 7. Zależności między zmiennymi a lekiem

In [ ]:
# Wiek vs Lek
plt.figure(figsize=(14, 8))
data.boxplot(column='Age', by='Drug_Name', ax=plt.gca(), rot=90)
plt.title('Rozkład wieku w zależności od leku')
plt.suptitle('')  # Usunięcie domyślnego tytułu
plt.xlabel('Lek')
plt.ylabel('Wiek')
plt.tight_layout()
plt.savefig('../results/age_by_drug.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Condition vs Drug_Name (heatmap)
condition_drug = pd.crosstab(data['Condition'], data['Drug_Name'])
plt.figure(figsize=(14, 8))
sns.heatmap(condition_drug, annot=True, fmt='d', cmap='YlOrRd')
plt.title('Zależność między schorzeniem a lekiem')
plt.xlabel('Lek')
plt.ylabel('Schorzenie')
plt.tight_layout()
plt.savefig('../results/condition_drug_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Gender vs Drug_Name
gender_drug = pd.crosstab(data['Gender'], data['Drug_Name'])
plt.figure(figsize=(14, 6))
gender_drug.plot(kind='bar', stacked=False)
plt.title('Rozkład płci w zależności od leku')
plt.xlabel('Płeć')
plt.ylabel('Liczba wystąpień')
plt.legend(title='Lek', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('../results/gender_drug.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Improvement Score vs Drug
plt.figure(figsize=(14, 8))
data.boxplot(column='Improvement_Score', by='Drug_Name', ax=plt.gca(), rot=90)
plt.title('Rozkład Improvement Score w zależności od leku')
plt.suptitle('')
plt.xlabel('Lek')
plt.ylabel('Improvement Score')
plt.tight_layout()
plt.savefig('../results/improvement_by_drug.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. Podsumowanie analizy

### Kluczowe obserwacje:
1. **Rozmiar zbioru**: {data.shape[0]} próbek, {data.shape[1]} cech
2. **Liczba klas**: {data['Drug_Name'].nunique()} różnych leków do klasyfikacji
3. **Brakujące wartości**: {data.isnull().sum().sum()} (brak)
4. **Zmienne numeryczne**: Age, Dosage_mg, Treatment_Duration_days, Improvement_Score
5. **Zmienne kategoryczne**: Gender, Condition, Drug_Name, Side_Effects

### Następne kroki:
- Preprocessing danych (encoding, normalizacja)
- Feature engineering
- Trenowanie modeli klasyfikacji
- Ewaluacja wyników